# TensorFlow examples - simplified

Hi there,

I do really love tensorflow [official documentation](https://www.tensorflow.org/versions/r0.7/how_tos/index.html), [API reference](https://www.tensorflow.org/versions/r0.7/api_docs/index.html) and [tutorials](https://www.tensorflow.org/versions/r0.7/tutorials/index.html), but I (personally) found [official code examples](https://github.com/tensorflow/tensorflow/tree/r0.7/tensorflow/examples/tutorials) to be annoyingly [complex](https://en.wikipedia.org/wiki/Cyclomatic_complexity). This file is an attempt to rewrite those examples so that anyone could read them from-bottom-to-top and get a (somewhat) full understanding of what is happening. Many parts are ommited or simplified just beacuse it seemed to me that it's more natural to structure code this way.

This could also be use as the code you look at while reading an official tutorial.

Warning: I'm a total tensorflow newby and this file reflects my newby's understanding of the subject - that could have little in common with reality :)

This is not a final version, though.

## Basic MINST Model (aka [MNIST For ML Beginners](https://www.tensorflow.org/versions/r0.7/tutorials/mnist/beginners/index.html))

In [37]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../data/MNIST_tf_data/", one_hot=True)

## placeholders - like theano.tensor.tensor()
## each time you run(vals=[vals]) some expression you need to specify
## values for the placeholders 
x_train = tf.placeholder(tf.float32, [None, 784])
y_train = tf.placeholder(tf.float32, [None, 10])

## variables - like theano.shared()
## retain values between calls 
## within a single Session() of execution
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y_pred = tf.nn.softmax(tf.matmul(x_train, W) + b)


## expression - can be _evaluated_
## (maps [vars] -> result, changes nothing - pure function)
cross_entropy = -tf.reduce_sum(y_train * tf.log(y_pred))
y_train_eq_pred = tf.equal(tf.argmax(y_train,1), tf.argmax(y_pred,1))
accuracy_exp = tf.reduce_mean(tf.cast(y_train_eq_pred, tf.float32))

## operation - can be _executed_ to updates values of Variables
## (maps [vars] -> None, updates state)
init_op = tf.initialize_all_variables()
train_step_op = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [41]:
## releases resources (memory, etc.) in the end of the block
with tf.Session() as s:
    s.run(init_op)
    for i in xrange(10000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        s.run(train_step_op, {x_train: batch_xs, y_train: batch_ys})
        
        if i % 1000 == 0:
            print i, s.run(accuracy_exp, 
                           feed_dict={x_train: mnist.test.images, y_train: mnist.test.labels})

0 0.4075
1000 0.916
2000 0.9127
3000 0.9228
4000 0.9203
5000 0.9181
6000 0.9208
7000 0.9185
8000 0.9229
9000 0.9212


### Links:

API:
* [tf.placeholder](https://www.tensorflow.org/versions/r0.7/api_docs/python/io_ops.html#placeholders)
* [tf.Variable](https://www.tensorflow.org/versions/r0.7/api_docs/python/state_ops.html#variables)
* [tf.Operation](https://www.tensorflow.org/versions/r0.7/api_docs/python/framework.html#Operation)
* [tf.Session](https://www.tensorflow.org/versions/r0.7/api_docs/python/client.html#session-management)

How-to
* https://www.tensorflow.org/versions/r0.7/get_started/basic_usage.html#overview
* https://www.tensorflow.org/versions/r0.7/how_tos/reading_data/index.html#feeding

### Ignore that:

this is not tensorflow, but just some utils

In [33]:
%%writefile ./mutils/config.py
## (ignore two cell below - that is not part of tf)
## If you want to run the code in this cell with without
## writing anywhere, just comment %%writefile and run the cell

class Immutable(type):
    def __init__(cls, name, bases, dct):
        def tmp(*a, **b):
            raise RuntimeError, ("This is config, don't create objects, just use it!")
        cls.__init__ = tmp

    def __setattr__(cls, key, value):
        if key != '__init__':
            raise RuntimeError, ("Configs are immutable! Don't do that.")
        else:
            super(Immutable, cls).__setattr__(key, value)

class BaseConfig:
    __metaclass__ = Immutable

Overwriting ./mutils/config.py


In [34]:
%%writefile ./mutils/record.py

from collections import namedtuple

def record(name, d):
    return namedtuple(name, d.keys())(**d)

Overwriting ./mutils/record.py


## MNIST Model (aka [TensorFlow Mechanics 101](https://www.tensorflow.org/versions/r0.7/tutorials/mnist/tf/index.html))

Original code: [mnist.py](https://github.com/tensorflow/tensorflow/blob/r0.7/tensorflow/examples/tutorials/mnist/mnist.py) and [fully_connected_feed.py](https://github.com/tensorflow/tensorflow/blob/r0.7/tensorflow/examples/tutorials/mnist/fully_connected_feed.py)

In [35]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist

In [36]:
from mutils.config import BaseConfig

## like namedtuple but with inheritance!
class BasicConfig(BaseConfig):    
    learning_rate = 0.01
    max_steps = 2000
    hidden1 = 128
    hidden2 = 32
    batch_size = 100
    train_dir = '/home/usman/data/tf_tmp'
    fake_data = False
       
## the cool thing about this config format is that 
## you can inhierit it for your specific case
## and override default values like:
## class CoolConfig(BasicConfig):
##     max_steps = 3000

config = BasicConfig

In [37]:
import math
from mutils.record import record

def hidden_layer(input_var, input_shape, hidden_units, name, activation=tf.nn.relu):
    ## in scope A the variable a would have absolute name = 'A/a'
    ## each layer definres it's own scope, variables are accessable 
    ## later via tf.get_variable('hidden1/w')
    with tf.name_scope(name):
        sigma = math.sqrt(float(hidden_units))
        init_val = tf.truncated_normal([input_shape, hidden_units], stddev=1.0/sigma)
        w = tf.Variable(initial_value=init_val, trainable=True, name='w')
        biases = tf.Variable(tf.zeros([hidden_units]), name='b')
        layer = activation(tf.matmul(input_var, w) + biases)
    return layer

def build_model(X_train_ps, y_train_ps, config):
    ## first two args are tf.placeholder's
    hidden1 = hidden_layer(X_train_ps, input_shape=mnist.IMAGE_PIXELS, hidden_units=100, 
                           name='hidden1')
    hidden2 = hidden_layer(hidden1, input_shape=100, hidden_units=100, 
                           name='hidden2')
    prob_y_pred = hidden_layer(hidden2, input_shape=100, hidden_units=10, 
                               name='logit', activation=lambda x: x)
    
    ## above code is eqvivalent to theano's: 
    ## hstack([labels.T, range(len(labels)).T])
    ## [ [0], [1], [2], [3], ...]
    indices = tf.expand_dims(tf.range(0, config.batch_size), 1)
    ## labels -> [ [2], [5], [2], [1], ...]
    y_train_ps_ = tf.expand_dims(y_train_ps, 1)
    ## [ [0, 2], [1, 5], ... ]
    y_train_ps_pairs = tf.concat(1, [indices, y_train_ps_])
    
    output_shape = tf.pack([config.batch_size, mnist.NUM_CLASSES])
    onehot_y_train = tf.sparse_to_dense(sparse_indices=y_train_ps_pairs,
                                        output_shape=output_shape, 
                                        sparse_values=1.0, default_value=0.0)
    
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
        prob_y_pred, onehot_y_train, name='xentropy')
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    ## score = sum(argmax(prob_y_pred) == y_i)
    correct = tf.nn.in_top_k(prob_y_pred, y_train_ps, 1)
    score = tf.reduce_sum(tf.cast(correct, tf.int32))
    
    optimizer = tf.train.GradientDescentOptimizer(config.learning_rate)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step)
    
    ## Add a scalar protobuf summary for the loss
    tf.scalar_summary("loss", loss)
    ## there must (!) be at least one summary before this call
    ## would cause an error otherwise
    summary_op = tf.merge_all_summaries()

    ## for variable values checkpoints 
    ## (automaticaly makes restarable copies on disk)
    saver = tf.train.Saver()
    
    ## to save/restore
    ## save_path = saver.save(sess, path)
    ## saver.restore(sess, path)
    
    ## see `record(..)` definition above
    return record('Model', {
        'train_op': train_op,
        'loss': loss,
        'score': score,
        'summary_op': summary_op,
        'saver': saver,
        'config': config,
    })

# model = build_model(config)

In [38]:
import time

def train(variables, model, data, verbose_every=100, validate_every=1000):
    with tf.Session() as s:
        ## writes provided summaries into ./dir/log
        writer = tf.train.SummaryWriter(model.config.train_dir, 
                                        graph_def=s.graph_def)
        
        ## operation that initializes all variables in graph
        s.run(tf.initialize_all_variables())
        
        for step in xrange(model.config.max_steps):
            start_time = time.time()
            data_batch = data.train.next_batch(model.config.batch_size, 
                                               model.config.fake_data)
            ## {X_train_ps: data_batch[0], y_train_ps: data_batch[1]}
            data_dict = dict(zip(variables, data_batch)) 
            ## train_op() -> None, model.loss() -> loss_value
            _, loss_value = s.run([model.train_op, model.loss], data_dict)
            duration = time.time() - start_time
            
            if verbose_every > 0 and step % verbose_every == 0:
                print('Step %d: loss = %.2f (%.3f sec)' % (step, 
                                                           loss_value, 
                                                           duration))
                
                ## failed to make following two lines work properly (?):
                # log_msg = s.run(model.summary_op, feed_dict=data_dict)
                # writer.add_summary(log_msg, step)
            
            # validate_every + after last iteration
            if (  (validate_every > 0 and step % validate_every == 0) or
                  (step == model.config.max_steps - 1) ):
                model.saver.save(s, model.config.train_dir, global_step=step)
                validation_sets_list = [ ('train', data.train), 
                                         ('test', data.test), 
                                         ('validate', data.validation)]
                for dataset_name, data_set in validation_sets_list:
                    n_batches = data_set.num_examples // model.config.batch_size
                    n_samples = n_batches*model.config.batch_size
                    score_sum = 0
                    for i in range(n_batches):
                        data_batch = data_set.next_batch(model.config.batch_size, 
                                                         model.config.fake_data)
                        data_dict = dict(zip(variables, data_batch)) 
                        score_sum += s.run(model.score, data_dict)
                    print('On "%s" dataset score is %.3f' % (dataset_name, 
                                                           float(score_sum) / n_samples))

In [39]:
def run():
    X_train_ps = tf.placeholder(tf.float32, shape=(config.batch_size, mnist.IMAGE_PIXELS))
    y_train_ps = tf.placeholder(tf.int32, shape=(config.batch_size))
    
    model = build_model(X_train_ps, y_train_ps, config)
    data = input_data.read_data_sets(config.train_dir, config.fake_data)
    
    train(variables=(X_train_ps, y_train_ps), 
          model=model, 
          data=data)
    
run()

Extracting /home/usman/data/tf_tmp/train-images-idx3-ubyte.gz
Extracting /home/usman/data/tf_tmp/train-labels-idx1-ubyte.gz
Extracting /home/usman/data/tf_tmp/t10k-images-idx3-ubyte.gz
Extracting /home/usman/data/tf_tmp/t10k-labels-idx1-ubyte.gz
Step 0: loss = 2.62 (0.108 sec)
On "train" dataset score is 0.067
On "test" dataset score is 0.062
On "validate" dataset score is 0.072
Step 100: loss = 1.34 (0.020 sec)
Step 200: loss = 0.76 (0.028 sec)
Step 300: loss = 0.60 (0.024 sec)
Step 400: loss = 0.54 (0.024 sec)
Step 500: loss = 0.43 (0.012 sec)
Step 600: loss = 0.61 (0.032 sec)
Step 700: loss = 0.42 (0.040 sec)
Step 800: loss = 0.29 (0.068 sec)
Step 900: loss = 0.47 (0.020 sec)
Step 1000: loss = 0.48 (0.012 sec)
On "train" dataset score is 0.893
On "test" dataset score is 0.899
On "validate" dataset score is 0.906
Step 1100: loss = 0.30 (0.772 sec)
Step 1200: loss = 0.15 (0.028 sec)
Step 1300: loss = 0.21 (0.040 sec)
Step 1400: loss = 0.39 (0.032 sec)
Step 1500: loss = 0.26 (0.028 sec

### Links

API:

* [tf.Graph](https://www.tensorflow.org/versions/r0.7/api_docs/python/framework.html#Graph)
* [tf.name_scope](https://www.tensorflow.org/versions/r0.7/api_docs/python/framework.html#name_scope) and [tf.variable_scope](https://www.tensorflow.org/versions/r0.7/api_docs/python/state_ops.html#variable_scope) and what is the [difference](http://stackoverflow.com/questions/34215746/difference-between-variable-scope-and-name-scope-in-tensorflow) between them?
* [tf.train.Optimizer](https://www.tensorflow.org/versions/r0.7/api_docs/python/train.html#optimizers)
* [Summary Operations](https://www.tensorflow.org/versions/r0.7/api_docs/python/train.html#summary-operations) (see [Protocol Buffers](https://en.wikipedia.org/wiki/Protocol_Buffers) aka protobuf) and [tf.train.SummaryWriter](https://www.tensorflow.org/versions/r0.7/api_docs/python/train.html#SummaryWriter)
* [tf.train.Saver](https://www.tensorflow.org/versions/r0.7/api_docs/python/state_ops.html#Saver)

## RNN Basics (aka [RNN tutorial](https://www.tensorflow.org/versions/r0.7/tutorials/recurrent/index.html))

Original [code](https://github.com/tensorflow/tensorflow/tree/r0.7/tensorflow/models/rnn/ptb)

In [28]:
from mutils.config import BaseConfig

class SmallConfig(BaseConfig):
    # MODEL HYPERPARAMS:
    num_layers = 2
    num_steps = 20 # maximum sequence length in a batch
    hidden_size = 200
    vocab_size = 10000
    dropout_prob = 0.0
    
    # TRANING:
    max_epoch = 13
    batch_size = 20
    learning_rate = 1.0
    grad_clip_norm = 5
    ## start slowing down learning starting from slowdown_epoch e.g. 
    ## real_learning_rate = learning_rate * lr_decay**(i - slowdown_epoch)
    slowdown_epoch = 4
    lr_decay = 0.5
    init_scale = 0.1 # std of weight initialization
    
    data_path = '../data/tf_langmodel/simple-examples/data'
    
## on evaluation step we need to process 
## data lines one-by-one; override settings
class EvaluationConfig(SmallConfig):
    batch_size = 1
    num_steps = 1

In [29]:
import tensorflow as tf
from tensorflow.models.rnn import rnn_cell
from tensorflow.models.rnn import seq2seq
from mutils.record import record

def build_model(X_placeholder, Y_placeholder, config, is_training=True):
    """
    X_placeholder and Y_placeholder
        - of shape [batch_size, num_steps]
    """
    is_using_dropout = is_training and config.dropout_prob > 0
    
    with tf.device("/cpu:0"):
        ## tf.get_variable - creates Variable() in current Scope() _if does not exit_
        embedding_matrix = tf.get_variable("embedding_matrix", [config.vocab_size, 
                                                                config.hidden_size])
        X_embedded = tf.nn.embedding_lookup(embedding_matrix, X_placeholder)
        ## - of shape [batch_size, num_steps, hidden_size]
    X_embedded_dropped = (tf.nn.dropout(X_embedded, 1-config.dropout_prob)
                          if is_using_dropout
                          else X_embedded)
    
    ## forget_bias > 0 - better results, but paper (ptb) uses forget_bias == 0
    lstm_cell = rnn_cell.BasicLSTMCell(config.hidden_size, forget_bias=0.0)
    lstm_cell_dropped = (rnn_cell.DropoutWrapper(lstm_cell, 
                                                 1-config.dropout_prob)
                         if is_using_dropout
                         else lstm_cell)
    cell = rnn_cell.MultiRNNCell([lstm_cell_dropped] * config.num_layers)
    
    ## in real-world scenario use tensorflow.models.rnn.rnn.py
    tutorial_mode = True
    
    if tutorial_mode:
        rnn_input = X_embedded_dropped
        outputs = []
        # [batch_size, hidden_size]
        init_state = cell.zero_state(config.batch_size, tf.float32)
        state = init_state
        with tf.variable_scope("RNN"):
            for time_step in range(config.num_steps):
                ## forces to use _same scope_ with _same variables_ as on t=0
                if time_step > 0:
                    tf.get_variable_scope().reuse_variables()
                (cell_output, state) = cell(rnn_input[:, time_step, :], state)
                outputs.append(cell_output)
    else:
        from tensorflow.models.rnn import rnn
        inputs = [tf.squeeze(input_slice, squeeze_dims=[1]) # shape[len, 1] -> shape[len]
                  for input_slice 
                  in tf.split(split_dim=1, num_split=num_steps, value=inputs)]
        init_state = cell.zero_state(config.batch_size, tf.float32)
        outputs, state = rnn.rnn(cell, inputs, initial_state=init_state)
    
    final_state = state

    rnn_output = tf.reshape(tf.concat(concat_dim=1, values=outputs), 
                            [-1, config.hidden_size])
    softmax_w = tf.get_variable("softmax_w", [config.hidden_size, config.vocab_size])
    softmax_b = tf.get_variable("softmax_b", [config.vocab_size])
    logits = tf.matmul(rnn_output, softmax_w) + softmax_b
    ## loss_arr: log-perplexity (cross-entropy) of shape [seq_len, ]
    loss_arr = seq2seq.sequence_loss_by_example(logits=[logits],
                                                targets=[tf.reshape(Y_placeholder, [-1])],
                                                weights=[tf.ones([config.batch_size * 
                                                                  config.num_steps])])
    loss = tf.reduce_sum(loss_arr) / config.batch_size

    if not is_training:
        return

    learning_rate = tf.Variable(0.0, trainable=False)
    model_weights = tf.trainable_variables()
    grads = tf.gradients(loss, model_weights)
    grads_clipped, norms = tf.clip_by_global_norm(grads, clip_norm=config.grad_clip_norm)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    ## we use custom compute_gradients() => use apply_gradients(), not minimize()
    train_op = optimizer.apply_gradients(zip(grads_clipped, model_weights))
    
    return record('Model', {
        'train_op': train_op,
        'loss': loss,
        'learning_rate': learning_rate,
        'config': config,
        'init_state': init_state,
        'final_state': final_state,
    })

def test():
    ## to avoid matrix re-definition errors in tf.get_variable()
    tf.reset_default_graph()
    
    config = SmallConfig
    X_p = tf.placeholder(tf.int32, [config.batch_size, config.num_steps])
    Y_p = tf.placeholder(tf.int32, [config.batch_size, config.num_steps])
    model = build_model(X_p, Y_p, config)
    
# test()

In [30]:
## if you're interested in dataparsing (I'm not), see:
from tensorflow.models.rnn.ptb import reader
import time
import numpy as np

def train(vars, model, data):
    ## if you use tf.Session().as_default()
    ##  - you replace tf.defaul_session object and can do
    ## `expression.eval()` instead of `s.run(expression)`
    ## BUT: resources won't be released after block ends
    
    ## if you do tf.Graph().as_default() - you need to 
    ## call build_model() _inside_ of this block, because
    ## model is in old default graph and call() is new graph
    with tf.Session() as s:
        s.run(tf.initialize_all_variables())
        
        ## for each: epoch
        for n_epoch in xrange(model.config.max_epoch):
            print 'epoch %d of %d' % (n_epoch, model.config.max_epoch)
            if n_epoch > model.config.slowdown_epoch:
                ## start slowing down learning starting from slowdown_epoch
                lr_decent = model.config.lr_decay ** (n_epoch - 
                                                      model.config.slowdown_epoch)
                s.run(tf.assign(model.learning_rate, model.config.learning_rate * lr_decent))
            dummy_op = tf.zeros([1, ], tf.int32) # does nothing
            ## for each: dataset
            for data_set, main_op, data_name in [(data.train, model.train_op, 'train'), 
                                                 (data.test, dummy_op, 'test')]:
                data_iterator = reader.ptb_iterator(data_set, 
                                                    model.config.batch_size, 
                                                    model.config.num_steps)
                
                ## verbose output
                n_batches = int(len(data_set) / 
                                model.config.batch_size / 
                                model.config.num_steps)
                start_time = time.time()
                ##
                
                state = s.run(model.init_state)
                loss = 0
                ## for each: batch
                for batch_n, batch_data in enumerate(data_iterator):
                    ## {x_placeholder: batch.x, .., init_state: state}
                    data_dict = dict(zip(vars, batch_data))
                    data_dict[model.init_state] = state
                    computation_plan = [model.loss, model.final_state, main_op]
                    batch_loss, state, _ = s.run(computation_plan, data_dict)
                    loss += batch_loss
                    
                    ## verbose output
                    if batch_n > 0 and batch_n % (n_batches / 10) == 0: # print 10 lines
                        time_diff = time.time() - start_time
                        batch_per_sec = float(batch_n) / time_diff
                        time_remained = (n_batches - batch_n) / batch_per_sec
                        print ('batch %d of %d for %.1f sec '
                               '(%.1f batch per second; %.1f sec remained)' % (
                                batch_n, n_batches, time_diff,
                                batch_per_sec, time_remained)
                              )
                        
                total_steps = model.config.num_steps*(batch_n+1)
                loss_per_one = loss / total_steps
                perplexity = np.exp(loss_per_one)

                print ("epoch %d, %s: unit loss"
                       "is %.3f, peplexity: %.3f" % (n_epoch, data_name, 
                                                     loss_per_one, perplexity))
                
            tick.tack() # end of epoch
        

In [31]:
from tensorflow.models.rnn.ptb import reader
from mutils.tick import tick

def run():
    tf.reset_default_graph()
    tick.tack(restart=True)
    
    config = SmallConfig
    eval_config = EvaluationConfig
    
    X_p = tf.placeholder(tf.int32, [config.batch_size, config.num_steps])
    Y_p = tf.placeholder(tf.int32, [config.batch_size, config.num_steps])

    print 'building model'
    ## to turn on GPU support install GPU version of tensorflow (see links)
    with tf.device('/gpu:0'):
        initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)
        with tf.variable_scope("model", reuse=None, initializer=initializer):
            train_model = build_model(X_p, Y_p, config)
        ## same model that shares weights (!) but
        ## does _not_ update anything and takes single input
        with tf.variable_scope("model", reuse=True, initializer=initializer):
            X_p_a = tf.placeholder(tf.int32, [eval_config.batch_size, eval_config.num_steps])
            Y_p_a = tf.placeholder(tf.int32, [eval_config.batch_size, eval_config.num_steps])
            apply_model = build_model(X_p_a, Y_p_a, eval_config, is_training=False)

    tick.tack()
    print 'reading data'
    raw_data = reader.ptb_raw_data(config.data_path)
    data = record('Data', {
            'train': raw_data[0],
            'test': raw_data[2]
        })
    
    tick.tack()
    print 'training'
    try:
        train((X_p, Y_p), train_model, data)
    except tf.errors.InvalidArgumentError as e:
        print 'ERROR: You probably dont have the specified device'
        print e

run()

building model
7.87sec (7.87sec total)
reading data
1.60sec (9.47sec total)
training
epoch 0 of 13
batch 232 of 2323 for 38.6 sec (6.0 batch per second; 347.9 sec remained)
batch 464 of 2323 for 74.8 sec (6.2 batch per second; 299.6 sec remained)
batch 696 of 2323 for 113.5 sec (6.1 batch per second; 265.4 sec remained)
batch 928 of 2323 for 152.5 sec (6.1 batch per second; 229.3 sec remained)
batch 1160 of 2323 for 191.7 sec (6.1 batch per second; 192.2 sec remained)
batch 1392 of 2323 for 226.4 sec (6.1 batch per second; 151.4 sec remained)
batch 1624 of 2323 for 257.3 sec (6.3 batch per second; 110.7 sec remained)
batch 1856 of 2323 for 293.3 sec (6.3 batch per second; 73.8 sec remained)
batch 2088 of 2323 for 330.9 sec (6.3 batch per second; 37.2 sec remained)
batch 2320 of 2323 for 369.1 sec (6.3 batch per second; 0.5 sec remained)
epoch 0, train: unit lossis 9.221, peplexity: 10110.956
batch 20 of 206 for 0.9 sec (22.2 batch per second; 8.4 sec remained)
batch 40 of 206 for 1.6 s

### Links

API:

* Understanding [tf.get_variable()](https://www.tensorflow.org/versions/r0.7/how_tos/variable_scope/index.html#understanding-tf-get-variable)
* RNN cell [source](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/rnn_cell.py)
* [tf.device](https://www.tensorflow.org/versions/r0.7/api_docs/python/framework.html#Graph.device)

GPU:
* https://www.tensorflow.org/versions/r0.7/how_tos/using_gpu/index.html#using-gpus
* https://www.tensorflow.org/versions/r0.7/get_started/os_setup.html#optional-linux-enable-gpu-support